# Soportes Tickets SQLAlchemy

In [ ]:
# Cargar paquetes e iniciar sesion a la BD
from datetime import datetime
from sqlalchemy.sql import func
from lib.database import SessionLocal
db = SessionLocal()

In [ ]:
# Cargar modelos
from plataforma_web.v1.autoridades.models import Autoridad
from plataforma_web.v1.distritos.models import Distrito
from plataforma_web.v1.domicilios.models import Domicilio
from plataforma_web.v1.funcionarios.models import Funcionario
from plataforma_web.v1.listas_de_acuerdos.models import ListaDeAcuerdo
from plataforma_web.v1.listas_de_acuerdos_acuerdos.models import ListaDeAcuerdoAcuerdo
from plataforma_web.v1.materias.models import Materia
from plataforma_web.v1.materias_tipos_juicios.models import MateriaTipoJuicio
from plataforma_web.v1.modulos.models import Modulo
from plataforma_web.v1.oficinas.models import Oficina
from plataforma_web.v1.permisos.models import Permiso
from plataforma_web.v1.roles.models import Rol
from plataforma_web.v1.sentencias.models import Sentencia
from plataforma_web.v1.soportes_categorias.models import SoporteCategoria
from plataforma_web.v1.soportes_tickets.models import SoporteTicket
from plataforma_web.v1.usuarios.models import Usuario
from plataforma_web.v1.usuarios_roles.models import UsuarioRol

In [ ]:
# Consultar categorias
soportes_categorias = db.query(SoporteCategoria.id, SoporteCategoria.nombre).\
    filter(SoporteCategoria.estatus == "A").\
    order_by(SoporteCategoria.nombre)
for id, nombre in soportes_categorias.limit(10).all():
    print(f"{id} - {nombre}")

In [ ]:
# Consultar oficinas
oficinas = db.query(Oficina.id, Oficina.clave, Oficina.descripcion_corta).\
    filter(Oficina.estatus == "A").\
    order_by(Oficina.clave)
for id, clave, descripcion_corta in oficinas.limit(10).all():
    print(f"{id} - {clave} - {descripcion_corta}")


In [ ]:
# Rango de tiempos
creado_desde = datetime(year=2022,month=2,day=21,hour=0,minute=0,second=0)
creado_hasta = datetime(year=2022,month=2,day=23,hour=23,minute=59,second=59)

# Subquery cantidades de tickets por categoria
precursor = db.query(SoporteTicket.soporte_categoria_id, func.count("*").label("cantidad")).\
    filter(SoporteTicket.creado >= creado_desde, SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    group_by(SoporteTicket.soporte_categoria_id).\
    subquery()

# Consultar categorias y cantidades de tickets
consulta = db.query(SoporteCategoria.nombre, precursor.c.cantidad).\
    filter(SoporteCategoria.estatus == "A").\
    outerjoin(precursor, SoporteCategoria.id == precursor.c.soporte_categoria_id)

# Top 10
renglon = 0
for nombre, cantidad in consulta.order_by(precursor.c.cantidad.desc()).all():
    if cantidad is not None:
        print(f"{nombre} - {cantidad}")
        renglon += 1
        if renglon > 10:
            break